In [0]:
%pip install crewai==0.85.0 crewai_tools==0.14.0 langchain_community==0.3.8
%pip install mlflow -U -q
dbutils.library.restartPython()

In [0]:
import os
import yaml
import sys
from crewai import Agent, Task, Crew


catalog = "fins_genai"
schema = "agents"

os.environ["DATABRICKS_API_KEY"] = dbutils.secrets.get("databricks_token_qyu", "qyu_rag_sp_token")
os.environ["DATABRICKS_API_BASE"] = (
    f"{dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()}/serving-endpoints"
)

# Set your Serper.dev API key for Google Search
os.environ["SERPER_API_KEY"] = dbutils.secrets.get("databricks_token_qyu", 'SerpApi')

# Define a Web Search Crew for Stock Symbols

In [0]:
from crewai_tools import SerperDevTool

# Initialize the search tool
search_tool = SerperDevTool()

In [0]:
from crewai import Agent
from langchain.chat_models import ChatOpenAI
from crewai import LLM

# Initialize the language model
llm = LLM(model="databricks/QYU_gpt4o_mini")

# Define the News Search Agent
news_search_agent = Agent(
    role="News Search Agent",
    goal="Find recent news articles related to the given stock ticker symbols {tickers} between {date1} and {date2}",
    backstory="An expert in financial news retrieval.",
    tools=[search_tool],
    verbose=True,
    llm=llm
)

# Define the Content Summarizer Agent
content_summarizer_agent = Agent(
    role="Content Summarizer Agent",
    goal="Summarize the content of the retrieved news articles. Highlight the positives, negatives, neutral points for each stock tickers",
    backstory="An expert in summarizing financial news.",
    verbose=True,
    llm=llm
)


In [0]:
from crewai import Task

# Task for searching news
search_news_task = Task(
    description="Search for news articles about the stock tickers {tickers} between {date1} and {date2}.",
    expected_output="news articles titles and content for each stock tickers", 
    agent=news_search_agent
)

# Task for summarizing news
summarize_news_task = Task(
    description="Review the news articles you found for each stock tickers, summarize the content of the news articles retrieved. highlights the positives, negatives, and neutral points for each stock tickers",
    expected_output="summarized news articles and highlights for each stock tickers",
    agent=content_summarizer_agent,
    context=[search_news_task]
)


## Assemble the crew

In [0]:
from crewai import Crew

# Create the crew
stock_news_crew = Crew(
    agents=[news_search_agent, content_summarizer_agent],
    tasks=[search_news_task, summarize_news_task],
    verbose=True
)


In [0]:
import mlflow

mlflow.crewai.autolog()

# Define the stock ticker symbol
ticker_symbol = "NVDA AAPL TSLA"
date1 = "Jan 24th, 2025"
date2 = "Feb 10th, 2025"

# Execute the crew
with mlflow.start_run() as run:
    stock_news_crew.kickoff(inputs={"tickers": ticker_symbol, "date1": date1, "date2": date2})
